In [1]:
'''Example script showing how to use stateful RNNs
to model long sequences efficiently.
'''
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential, Model
from keras.layers import Dense, LSTM, Flatten, Input
import pandas as pd
import scipy.io as sio
import os

# since we are using stateful rnn tsteps can be set to 1
tsteps = 1
batch_size = 1
epochs = 25

# number of elements ahead that are used to make the prediction
lahead = 1

#raw_path = "/home/ihsan/Documents/thesis_generator/results/devin/to_process/" #needs the absolute path, no tildes! 
#processed_path = "/home/ihsan/Documents/thesis_generator/results/devin"
train_path = "/home/ihsan/Documents/thesis_models/train/"
test_path = "/home/ihsan/Documents/thesis_models/test/"

Using Theano backend.


In [2]:
#11 input columns 
#4 output columns. 



#load data multiple times. 
data_filenames = os.listdir(train_path + "data")
#print("before sorting, data_filenames: {}".format(data_filenames))
data_filenames.sort()
#print("after sorting, data_filenames: {}".format(data_filenames))


label_filenames = os.listdir(train_path + "label")
label_filenames.sort()
#print("label_filenames: {}".format(data_filenames))
combined_filenames = zip(data_filenames,label_filenames)
print(combined_filenames)
assert len(data_filenames) == len(label_filenames)

'''print('Creating Model...')
model = Sequential()
model.add(LSTM(64,
               input_shape = (None,11),
               return_sequences=True,
               stateful=False))
model.add(LSTM(64,return_sequences=False,stateful=False))
#model.add(Flatten())
#model.add(Dense(128))
model.add(Dense(4))
model.compile(loss='mse', optimizer='rmsprop')

model.summary()'''
#define the model first
a = Input(shape=(batch_size,None,11))
b = LSTM(64,return_sequences=True)(a)
c = LSTM(64,return_sequences=False)(b)
out = Dense(4)(c)

model = Model(input=a,outputs=out)

print("Inputs: {}".format(model.input_shape))
print ("Outputs: {}".format(model.output_shape))
#print ("Actual input: {}".format(data.shape))
#print ("Actual output: {}".format(target.shape))



[('sequence_1a_1.npy', 'sequence_1a_1_label_.npy'), ('sequence_1a_2.npy', 'sequence_1a_2_label_.npy'), ('sequence_1a_3.npy', 'sequence_1a_3_label_.npy'), ('sequence_2a_1.npy', 'sequence_2a_1_label_.npy'), ('sequence_2a_2.npy', 'sequence_2a_2_label_.npy'), ('sequence_2a_3.npy', 'sequence_2a_3_label_.npy'), ('sequence_3a_1.npy', 'sequence_3a_1_label_.npy'), ('sequence_3a_2.npy', 'sequence_3a_2_label_.npy'), ('sequence_3a_3.npy', 'sequence_3a_3_label_.npy'), ('sequence_4a_1.npy', 'sequence_4a_1_label_.npy'), ('sequence_4a_2.npy', 'sequence_4a_2_label_.npy'), ('sequence_4a_3.npy', 'sequence_4a_3_label_.npy')]


ValueError: Input 0 is incompatible with layer lstm_1: expected ndim=3, found ndim=4

In [3]:
print('loading data...')

#tuples 
for files in combined_filenames: 
    data_load_path = train_path + '/data/' + files[0]
    label_load_path = train_path + '/label/' + files[1]
    #print("data/label load path: {} \n {}".format(data_load_path,label_load_path))
    train_array = np.load(data_load_path)
    label_array = np.load(label_load_path)
    train_array = np.reshape(train_array,(1,train_array.shape[0],train_array.shape[1]))
    label_array = np.reshape(label_array,(1,label_array.shape[0],label_array.shape[1]))
    #print("data/label shape: {} \n {}".format(train_array.shape,label_array.shape))

    print('Training')
    for i in range(epochs):
        print('Epoch', i, '/', epochs)

    # Note that the last state for sample i in a batch will
    # be used as initial state for sample i in the next batch.
    # Thus we are simultaneously training on batch_size series with
    # lower resolution than the original series contained in cos.
    # Each of these series are offset by one step and can be
    # extracted with cos[i::batch_size].

        model.fit(train_array, label_array,
              batch_size=batch_size,
              epochs=2,
              verbose=1,
              shuffle=False)


loading data...
Training
Epoch 0 / 25


ValueError: Error when checking target: expected dense_1 to have 2 dimensions, but got array with shape (1, 11520, 5)

In [ ]:
print('Predicting')
predicted_output = model.predict(cos, batch_size=batch_size)

print('Plotting Results')
plt.subplot(2, 1, 1)
plt.plot(expected_output)
plt.title('Expected')
plt.subplot(2, 1, 2)
plt.plot(predicted_output)
plt.title('Predicted')
plt.show()